# Azure OpenAI bring your own data api test

Ingestion

### REST API

https://learn.microsoft.com/en-us/azure/ai-services/openai/reference#completions-extensions

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import requests
import json
import os

AOAI_KEY = os.environ['AOAI_KEY']
AOAI_ENDPOINT = os.environ['AOAI_ENDPOINT']
#MODEL = 'gpt-35-turbo-16k'
MODEL = 'gpt-35-turbo'
#MODEL = 'gpt-4'

EMBED_AOAI_KEY = os.environ['JP_AOAI_KEY']
EMBED_AOAI_ENDPOINT = os.environ['JP_AOAI_ENDPOINT']

EMBED_MODEL = 'text-embedding-ada-002'
VERSION = '2023-08-01-preview'

In [4]:
AOAI_ENDPOINT = os.environ['JP_AOAI_ENDPOINT']
AOAI_KEY = os.environ['JP_AOAI_KEY']

STORAGE_CONNECTION_STRING = os.environ['STORAGE_CONNSTR']

AZSCH_ENDPOINT = os.getenv("AZSCH_ENDPOINT")  
AZSCH_KEY = os.getenv("AZSCH_KEY")  

AOAI_EMBEDIING_ENDPOINT=f'{EMBED_AOAI_ENDPOINT}openai/deployments/{EMBED_MODEL}/embeddings?api-version=2023-05-15'

### NOTE

`dataRefreshIntervalInMinutes = 0` -  If you want to run a single ingestion job without a schedule, set this parameter to 0.

In [23]:
import requests
import json
import sseclient
import os
import time
from datetime import datetime
import sys

def ingest_request(job, container):
    AOAI_BYOD_ENDPOINT = f'{AOAI_ENDPOINT}openai/extensions/on-your-data/ingestion-jobs/{job}?api-version=2023-10-01-preview'

    try:
        url = AOAI_BYOD_ENDPOINT
        headers = {
            'Content-Type': 'application/json',
            'api-key': AOAI_KEY,
            'searchServiceEndpoint': AZSCH_ENDPOINT,
            'searchServiceAdminKey': AZSCH_KEY,
            'storageConnectionString': STORAGE_CONNECTION_STRING,
            'storageContainer': container,
            'embeddingEndpoint': AOAI_EMBEDIING_ENDPOINT,
            'embeddingKey': AOAI_KEY
        }
        body = {
            'dataRefreshIntervalInMinutes': 0
        }


        request = requests.put(url, headers=headers, json=body)

        return json.loads(request.text)
    except Exception as ex:
        return ex
    

In [24]:
ingest_request('appsvc', 'data')

{'id': 'appsvc',
 'dataRefreshIntervalInMinutes': 0,
 'completionAction': 'cleanUpAssets',
 'status': 'notRunning',
 'warnings': [],
 'progress': {'stageProgress': [{'name': 'Preprocessing',
    'totalItems': 0,
    'processedItems': 0},
   {'name': 'Indexing', 'totalItems': 0, 'processedItems': 0}]}}

In [25]:
def ingest_list():
    AOAI_BYOD_ENDPOINT = f'{AOAI_ENDPOINT}openai/extensions/on-your-data/ingestion-jobs?api-version=2023-10-01-preview'

    try:
        url = AOAI_BYOD_ENDPOINT
        headers = {
            'Content-Type': 'application/json',
            'api-key': AOAI_KEY,
        }


        request = requests.get(url, headers=headers)

        return json.loads(request.text)
    except Exception as ex:
        return ex

In [26]:
ingest_list()

{'value': [{'id': 'appsvc',
   'dataRefreshIntervalInMinutes': 0,
   'completionAction': 'cleanUpAssets',
   'status': 'notRunning',
   'warnings': [],
   'progress': {'stageProgress': [{'name': 'Preprocessing',
      'totalItems': 0,
      'processedItems': 0},
     {'name': 'Indexing', 'totalItems': 0, 'processedItems': 0}]}}]}

In [27]:
def ingest_status(job):
    AOAI_BYOD_ENDPOINT = f'{AOAI_ENDPOINT}openai/extensions/on-your-data/ingestion-jobs?api-version=2023-10-01-preview'
    
    try:
        url = AOAI_BYOD_ENDPOINT
        headers = {
            'Content-Type': 'application/json',
            'api-key': AOAI_KEY,
        }


        request = requests.get(url, headers=headers)

        return json.loads(request.text)
    except Exception as ex:
        return ex
    

In [29]:
ingest_status('appsvc')

{'id': 'appsvc',
 'dataRefreshIntervalInMinutes': 0,
 'completionAction': 'cleanUpAssets',
 'status': 'running',
 'warnings': ["This ADLS Gen2 indexer maps the property 'metadata_storage_path' to the index key, which may not reindex documents if directories are renamed. Update the 'LastModified' timestamps for all the blobs in the directory to ensure they get reindexed.  (1 item(s) impacted)",
  "Truncated extracted text to '65536' characters.  (1 item(s) impacted)"],
 'progress': {'stageProgress': [{'name': 'Preprocessing',
    'totalItems': 185,
    'processedItems': 145},
   {'name': 'Indexing', 'totalItems': 0, 'processedItems': 0}]}}